# 1. Data Preparation

## Imports

In [1]:
import os
import shutil
import pandas as pd

## Feasibilty filtering


In [2]:
# read solu file to get all feasible instances
def get_solved_instances(solu_path):
    solved = set()
    with open(solu_path, "r") as f:
        for line in f:
            parts = line.strip().split()
            if len(parts) >= 3 and parts[0] in ("=opt=", "=best="):
                solved.add(parts[1].lower())
    return solved

# copy solved .mps.gz file
def copy_solved_mps_files(src_folder, dst_folder, solved_instances):
    os.makedirs(dst_folder, exist_ok=True)
    copied_files = []
    for file in os.listdir(src_folder):
        if file.endswith(".mps.gz"):
            instance = file.replace(".mps.gz", "").lower()
            if instance in solved_instances:
                src_path = os.path.join(src_folder, file)
                dst_path = os.path.join(dst_folder, file)
                shutil.copy2(src_path, dst_path)
                copied_files.append(file)
    print(f"have copied {len(copied_files)} feasible files into {dst_folder}")
    return copied_files

In [4]:
# Paths
benchmark_folder = "../data/raw/benchmark/"
solu_file = "../data/raw/miplib2017-v35.solu.txt"
solved_folder = "../data/raw/benchmark_solved/"

# Execute copying process
solved_names = get_solved_instances(solu_file)
copied_files = copy_solved_mps_files(benchmark_folder, solved_folder, solved_names)

have copied 233 feasible files into ../data/raw/benchmark_solved/


In [5]:
# Preview of copied files
df_copied = pd.DataFrame(copied_files, columns=["mps_file"])
df_copied.head()

,mps_file
0,30n20b8.mps.gz
1,50v-10.mps.gz
2,academictimetablesmall.mps.gz
3,air05.mps.gz
4,app1-1.mps.gz


## Prepare for starter